---
## Step 1 — GPU Check & Install Dependencies

---
## Step 1 — GPU Check & Install Dependencies

In [ ]:
#@title 1a. Verify GPU
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv
!nvidia-smi | grep 'CUDA Version'

In [ ]:
#@title 1b. Install packages
!pip install -q onnxruntime-gpu>=1.16.0 onnx pycocotools>=2.0.7 \
    PyYAML>=6.0 matplotlib>=3.7.0 pandas>=2.0.0 opencv-python-headless>=4.8.0

import onnxruntime as ort
print(f"\n\u2705 ONNX Runtime {ort.__version__}")
print(f"   Providers: {ort.get_available_providers()}")
assert 'CUDAExecutionProvider' in ort.get_available_providers(), \
    "\u274c CUDAExecutionProvider not found! Runtime > Change runtime type > GPU"

---
## Step 2 — Clone Repository

This pulls **everything** — code, images, annotations, AND all ONNX models (via Git LFS).

In [ ]:
#@title 2. Clone repo with LFS models
import os

REPO_URL = "https://github.com/Sasankamadura/FYP-Final-Testing.git"
WORKSPACE = "/content/FYP-Final-Testing"

if os.path.exists(WORKSPACE):
    print(f"Repo already exists at {WORKSPACE}, pulling latest...")
    !cd "{WORKSPACE}" && git pull && git lfs pull
else:
    # Install Git LFS and clone
    !git lfs install
    !git clone "{REPO_URL}" "{WORKSPACE}"

os.chdir(WORKSPACE)
print(f"\n\u2705 Working directory: {os.getcwd()}")

# Verify LFS files were downloaded (not just pointers)
!git lfs ls-files | head -5
onnx_count = sum(1 for r, d, fs in os.walk('.') for f in fs if f.endswith('.onnx'))
print(f"\n\u2705 ONNX model files: {onnx_count}")

---
## Step 3 — Verify Workspace

In [ ]:
#@title 3. Check everything is in place
import os, yaml
os.chdir(WORKSPACE)

print("\u2501" * 60)
print("  DIRECTORY CHECK")
print("\u2501" * 60)
for d in ['eval', 'eval/utils',
          'Checkpoints - Baseline_Visdrone2019',
          'Checkpoints - After improvements',
          'VisDrone Val image set',
          'VisDrone test image set/images']:
    print(f"  {'\u2705' if os.path.isdir(d) else '\u274c'} {d}")

ann = 'VisDrone Val image set/annotations_VisDrone_val.json'
print(f"\n  Annotations: {'\u2705' if os.path.exists(ann) else '\u274c'}")

val_imgs = [f for f in os.listdir('VisDrone Val image set') if f.endswith(('.jpg','.png'))]
test_imgs = [f for f in os.listdir('VisDrone test image set/images') if f.endswith(('.jpg','.png'))]
print(f"  Val images:  {len(val_imgs)}")
print(f"  Test images: {len(test_imgs)}")

print(f"\n\u2501" * 60)
print("  MODEL CHECKPOINT VERIFICATION")
print("\u2501" * 60)
with open('eval/config.yaml') as f:
    config = yaml.safe_load(f)

all_ok = True
for key, mcfg in config['models'].items():
    exists = os.path.exists(mcfg['path'])
    size_str = f"({os.path.getsize(mcfg['path'])/(1024*1024):.1f} MB)" if exists else ""
    if not exists: all_ok = False
    print(f"  {'\u2705' if exists else '\u274c MISSING'} {mcfg['name']:<40} {size_str}")

print()
if all_ok:
    print("\u2705 All models found! Ready to run.")
else:
    print("\u274c Some models missing. Try: git lfs pull")

---
## Step 4 — Run Validation (COCO mAP)

Computes mAP@50, mAP@50:95, AP-small/medium/large, per-class AP for all models.

Has **resume support** — if Colab disconnects, re-run and it skips completed models.

> ⏱ ~30–60 min for all models on T4

In [ ]:
#@title 4a. Run validation — ALL models
os.chdir(WORKSPACE)
!python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 4b. (Optional) Validate a SINGLE model first
# Uncomment one line to quick-test:

# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model baseline_rtdetr_r18
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model gnconv_p2_crr
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model efficientnet_b2_p2

---
## Step 5 — Run Latency Benchmark

Measures mean/median/P95/P99 latency, FPS, GPU memory, model size, parameter count.

> ⏱ ~10–20 min for all models

In [ ]:
#@title 5a. Benchmark — ALL models
os.chdir(WORKSPACE)
!python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 5b. (Optional) Benchmark with end-to-end timing
# !python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}" --e2e

---
## Step 6 — Generate Report

In [ ]:
#@title 6. Generate tables, CSVs, and plots
os.chdir(WORKSPACE)
!python eval/generate_report.py --config eval/config.yaml --workspace "{WORKSPACE}"

---
## Step 7 — View Results

In [ ]:
#@title 7a. Accuracy Comparison
import pandas as pd
from IPython.display import display
os.chdir(WORKSPACE)

csv_path = 'eval/results/reports/accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}',
        'mAP_small': '{:.4f}', 'mAP_medium': '{:.4f}', 'mAP_large': '{:.4f}',
        'delta_mAP_50': '{:+.4f}'
    }).background_gradient(subset=['mAP_50'], cmap='Greens'))
else:
    print('Run Steps 4 + 6 first.')

In [ ]:
#@title 7b. Speed vs Accuracy
csv_path = 'eval/results/reports/speed_accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}', 'mAP_small': '{:.4f}',
        'latency_ms': '{:.2f}', 'fps': '{:.1f}', 'size_mb': '{:.1f}',
        'efficiency_score': '{:.2f}'
    }).background_gradient(subset=['efficiency_score'], cmap='YlGn'))
else:
    print('Run Steps 4 + 5 + 6 first.')

In [ ]:
#@title 7c. Per-Class AP@50
csv_path = 'eval/results/reports/per_class_ap50.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    ap_cols = [c for c in df.columns if c.startswith('AP50_')]
    display(df.style.format({c: '{:.4f}' for c in ap_cols})
            .background_gradient(subset=ap_cols, cmap='YlOrRd', vmin=0, vmax=0.7))

In [ ]:
#@title 7d. Display All Plots
from IPython.display import Image as IPImage, display
import glob

plots_dir = 'eval/results/reports/plots'
if os.path.isdir(plots_dir):
    for pf in sorted(glob.glob(os.path.join(plots_dir, '*.png'))):
        print(f"\n\u2500\u2500 {os.path.basename(pf)} \u2500\u2500")
        display(IPImage(filename=pf, width=900))
else:
    print('Run Step 6 first.')

In [ ]:
#@title 7e. Raw JSON results summary
import json
results_root = 'eval/results'
for gpu_dir in sorted(d for d in os.listdir(results_root)
                      if os.path.isdir(os.path.join(results_root, d)) and d.startswith('GPU_')):
    val_file = os.path.join(results_root, gpu_dir, 'validation', 'all_validation_results.json')
    if os.path.exists(val_file):
        with open(val_file) as f: val = json.load(f)
        print(f"\n\u2501 {gpu_dir} \u2014 {len(val)} models")
        for k, v in val.items():
            m = v['metrics']
            print(f"  {v['name']:<40} mAP@50={m['mAP_50']:.4f}  AP-S={m['mAP_small']:.4f}")

    bench_file = os.path.join(results_root, gpu_dir, 'benchmark', 'benchmark_results.json')
    if os.path.exists(bench_file):
        with open(bench_file) as f: bench = json.load(f)
        print(f"\n  Benchmark:")
        for k, v in bench.items():
            print(f"  {v['name']:<40} {v['mean_latency_ms']:.2f}ms  FPS={v['fps']:.1f}")

---
## Step 8 — Download Results

In [ ]:
#@title 8a. Download as zip
import shutil
from google.colab import files

zip_path = '/content/eval_results'
shutil.make_archive(zip_path, 'zip', os.path.join(WORKSPACE, 'eval/results'))
size_mb = os.path.getsize(zip_path + '.zip') / (1024*1024)
print(f"\u2705 {zip_path}.zip ({size_mb:.1f} MB)")
files.download(f'{zip_path}.zip')

In [ ]:
#@title 8b. (Optional) Push Colab results back to GitHub
# os.chdir(WORKSPACE)
# !git add eval/results/
# !git commit -m "Add Colab T4 GPU evaluation results"
# !git push origin main
#
# # For private repos use a Personal Access Token:
# # !git push https://<YOUR_TOKEN>@github.com/Sasankamadura/FYP-Final-Testing.git main

---
## 💡 Tips

| Tip | Details |
|-----|---------|
| **Resume support** | If Colab disconnects, re-run Step 4 — already-evaluated models are skipped |
| **Single model test** | Use `--model baseline_rtdetr_r18` to test one model quickly |
| **E2E benchmark** | Add `--e2e` to also measure preprocess + postprocess time |
| **Cross-GPU comparison** | Your repo has RTX 4050 results; after Colab run, report auto-generates cross-GPU table |
| **Colab Pro** | A100 gives ~3–5x faster benchmarks than T4 |

| Issue | Fix |
|-------|-----|
| `CUDAExecutionProvider not available` | Runtime → Change runtime type → GPU |
| ONNX files are tiny (~1KB) | LFS pointers not downloaded. Run `!git lfs pull` |
| `Session crashed` | Re-run Step 4 (resume skips done models) |

In [ ]:
#@title 1a. Verify GPU is available
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv
print()
!nvidia-smi | grep 'CUDA Version'

In [ ]:
#@title 1b. Install required packages
!pip install -q onnxruntime-gpu>=1.16.0 onnx pycocotools>=2.0.7 \
    PyYAML>=6.0 matplotlib>=3.7.0 pandas>=2.0.0 opencv-python-headless>=4.8.0

# Verify GPU provider is available
import onnxruntime as ort
print(f"\n\u2705 ONNX Runtime {ort.__version__}")
print(f"   Providers: {ort.get_available_providers()}")
assert 'CUDAExecutionProvider' in ort.get_available_providers(), \
    "\u274c CUDAExecutionProvider not found! Go to Runtime > Change runtime type > GPU"

---
## Step 2 — Clone Your GitHub Repository

This pulls all code, images, annotations, and previous results.

The ONNX model files are `.gitignore`d (too large for GitHub), so we'll get those from Google Drive in Step 3.

In [ ]:
#@title 2. Clone the repo
import os

REPO_URL = "https://github.com/Sasankamadura/FYP-Final-Testing.git"
WORKSPACE = "/content/FYP-Final-Testing"

if os.path.exists(WORKSPACE):
    print(f"Repo already cloned at {WORKSPACE}, pulling latest...")
    !cd "{WORKSPACE}" && git pull
else:
    !git clone "{REPO_URL}" "{WORKSPACE}"

os.chdir(WORKSPACE)
print(f"\n\u2705 Working directory: {os.getcwd()}")
print(f"   Contents: {os.listdir('.')}")

---
## Step 3 — Get ONNX Checkpoints from Google Drive

Your `.onnx` files are gitignored (~1.8 GB total). You need to upload them to Google Drive first.

### 📁 How to prepare your Drive

On your PC, copy **only** the two checkpoint folders to Google Drive:
```
My Drive/
  FYP-Checkpoints/
    Checkpoints - Baseline_Visdrone2019/
      RT-DETR Resnet 18/
        base_rtdetr.onnx
    Checkpoints - After improvements/
      1-P2-P3 fusion/
        model_p2p3_F.onnx
      2-Query IMP/
        model (1).onnx
      ... (all other subfolders with .onnx files)
```

Then run the cell below.

In [ ]:
#@title 3a. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 3b. Copy ONNX checkpoints into the cloned repo
import shutil
import os

# ===========================================================
# EDIT THIS to match YOUR Google Drive folder path
# ===========================================================
DRIVE_CHECKPOINTS = "/content/drive/MyDrive/FYP-Checkpoints"

# The two folders to copy
folders_to_copy = [
    "Checkpoints - Baseline_Visdrone2019",
    "Checkpoints - After improvements",
]

os.chdir(WORKSPACE)

for folder in folders_to_copy:
    src = os.path.join(DRIVE_CHECKPOINTS, folder)
    dst = os.path.join(WORKSPACE, folder)

    if not os.path.exists(src):
        print(f"\u274c Source not found: {src}")
        print(f"   Check your DRIVE_CHECKPOINTS path!")
        continue

    if os.path.exists(dst):
        print(f"\u2705 Already exists: {folder} (skipping copy)")
    else:
        print(f"\u23f3 Copying {folder} ...")
        shutil.copytree(src, dst)
        print(f"\u2705 Copied: {folder}")

# Verify ONNX files are in place
onnx_count = 0
for root, dirs, files in os.walk(WORKSPACE):
    for f in files:
        if f.endswith('.onnx'):
            onnx_count += 1
print(f"\n\u2705 Total ONNX model files found: {onnx_count}")

---
## Step 4 — Verify Workspace Structure
Make sure all checkpoints, images, and annotations are accessible.

In [ ]:
#@title 4. Full workspace verification
import os, yaml
os.chdir(WORKSPACE)

# --- Check key directories ---
print("\u2501" * 60)
print("  DIRECTORY CHECK")
print("\u2501" * 60)
expected_dirs = [
    'eval',
    'eval/utils',
    'Checkpoints - Baseline_Visdrone2019',
    'Checkpoints - After improvements',
    'VisDrone Val image set',
    'VisDrone test image set/images',
]
for d in expected_dirs:
    exists = os.path.isdir(d)
    print(f"  {'\u2705' if exists else '\u274c'} {d}")

# --- Check annotations ---
ann = 'VisDrone Val image set/annotations_VisDrone_val.json'
print(f"\n  Annotations: {'\u2705' if os.path.exists(ann) else '\u274c'} {ann}")

# --- Count images ---
val_imgs = [f for f in os.listdir('VisDrone Val image set') if f.endswith(('.jpg','.png'))]
test_imgs = [f for f in os.listdir('VisDrone test image set/images') if f.endswith(('.jpg','.png'))]
print(f"  Val images: {len(val_imgs)}")
print(f"  Test images: {len(test_imgs)}")

# --- Load config & check each model ---
print(f"\n\u2501" * 60)
print("  MODEL CHECKPOINT VERIFICATION")
print("\u2501" * 60)
with open('eval/config.yaml') as f:
    config = yaml.safe_load(f)

all_ok = True
for key, mcfg in config['models'].items():
    exists = os.path.exists(mcfg['path'])
    status = '\u2705' if exists else '\u274c MISSING'
    size_str = ""
    if exists:
        size_mb = os.path.getsize(mcfg['path']) / (1024*1024)
        size_str = f"({size_mb:.1f} MB)"
    else:
        all_ok = False
    print(f"  {status} {mcfg['name']:<40} {size_str}")

print()
if all_ok:
    print("\u2705 All models found! Ready to run evaluation.")
else:
    print("\u274c Some models are missing. Check your Google Drive path in Step 3b.")

---
## Step 5 — Run Validation (COCO mAP)

Runs inference on the VisDrone val set and computes:
- **mAP@50**, **mAP@50:95**
- **AP-small**, **AP-medium**, **AP-large**
- **Per-class AP@50** for all 10 VisDrone classes

Results are saved per-model with **resume support** — if Colab disconnects, re-run and it skips already-evaluated models.

> ⏱ ~30–60 min for all 21 models on T4

In [ ]:
#@title 5a. Run validation for ALL models
os.chdir(WORKSPACE)
!python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 5b. (Optional) Run validation for a SINGLE model (quick test)
# Uncomment one of these to test a single model first:

# os.chdir(WORKSPACE)
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model baseline_rtdetr_r18
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model gnconv_p2_crr
# !python eval/run_validation.py --config eval/config.yaml --workspace "{WORKSPACE}" --model efficientnet_b2_p2

---
## Step 6 — Run Latency Benchmark

Measures per-model:
- **Mean / Median / P95 / P99 latency** (ms)
- **FPS** (frames per second)
- **GPU memory** usage
- **Model size** (MB) and **parameter count**

> ⏱ ~10–20 min for all models (50 warmup + 200 measure iterations each)

In [ ]:
#@title 6a. Run benchmark for ALL models
os.chdir(WORKSPACE)
!python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}"

In [ ]:
#@title 6b. (Optional) Also run end-to-end benchmark (includes preprocessing)
# os.chdir(WORKSPACE)
# !python eval/run_benchmark.py --config eval/config.yaml --workspace "{WORKSPACE}" --e2e

---
## Step 7 — Generate Report

Aggregates validation + benchmark results into:
- **Accuracy comparison table** (CSV)
- **Per-class AP@50 table** (CSV)
- **Speed vs Accuracy comparison** (CSV)
- **3-dec vs 6-dec comparison**
- **Plots**: mAP bar chart, Pareto scatter, AP by object size, per-class heatmap, FPS chart

In [ ]:
#@title 7. Generate full report
os.chdir(WORKSPACE)
!python eval/generate_report.py --config eval/config.yaml --workspace "{WORKSPACE}"

---
## Step 8 — View Results Inline

In [ ]:
#@title 8a. Accuracy Comparison Table
import pandas as pd
from IPython.display import display
os.chdir(WORKSPACE)

csv_path = 'eval/results/reports/accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"\u2501" * 80)
    print("  ACCURACY COMPARISON")
    print(f"\u2501" * 80)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}',
        'mAP_small': '{:.4f}', 'mAP_medium': '{:.4f}', 'mAP_large': '{:.4f}',
        'delta_mAP_50': '{:+.4f}'
    }).background_gradient(subset=['mAP_50'], cmap='Greens'))
else:
    print("\u274c Not found. Run Steps 5 + 7 first.")

In [ ]:
#@title 8b. Speed vs Accuracy Table
csv_path = 'eval/results/reports/speed_accuracy_comparison.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"\u2501" * 80)
    print("  SPEED vs ACCURACY")
    print(f"\u2501" * 80)
    display(df.style.format({
        'mAP_50': '{:.4f}', 'mAP_50_95': '{:.4f}', 'mAP_small': '{:.4f}',
        'latency_ms': '{:.2f}', 'fps': '{:.1f}', 'size_mb': '{:.1f}',
        'efficiency_score': '{:.2f}'
    }).background_gradient(subset=['efficiency_score'], cmap='YlGn'))
else:
    print("\u274c Not found. Run Steps 5 + 6 + 7 first.")

In [ ]:
#@title 8c. Per-Class AP@50
csv_path = 'eval/results/reports/per_class_ap50.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("PER-CLASS AP@50")
    # Color each AP column
    ap_cols = [c for c in df.columns if c.startswith('AP50_')]
    display(df.style.format({c: '{:.4f}' for c in ap_cols})
            .background_gradient(subset=ap_cols, cmap='YlOrRd', vmin=0, vmax=0.7))
else:
    print("\u274c Not found. Run Steps 5 + 7 first.")

In [ ]:
#@title 8d. Display All Generated Plots
from IPython.display import Image as IPImage, display
import glob

plots_dir = 'eval/results/reports/plots'
if os.path.isdir(plots_dir):
    plot_files = sorted(glob.glob(os.path.join(plots_dir, '*.png')))
    print(f"\u2705 Found {len(plot_files)} plots\n")
    for pf in plot_files:
        print(f"\u2500\u2500 {os.path.basename(pf)} \u2500\u2500")
        display(IPImage(filename=pf, width=900))
        print()
else:
    print("\u274c No plots found. Run Step 7 first.")

In [ ]:
#@title 8e. Quick look at raw JSON results
import json

# Find the GPU results folder for this Colab session
results_root = 'eval/results'
gpu_dirs = [d for d in os.listdir(results_root)
            if os.path.isdir(os.path.join(results_root, d)) and d.startswith('GPU_')]
print(f"GPU result folders: {gpu_dirs}\n")

for gpu_dir in gpu_dirs:
    # Validation summary
    val_file = os.path.join(results_root, gpu_dir, 'validation', 'all_validation_results.json')
    if os.path.exists(val_file):
        with open(val_file) as f:
            val = json.load(f)
        print(f"\u2501 {gpu_dir} \u2014 Validation: {len(val)} models evaluated")
        for k, v in val.items():
            m = v['metrics']
            print(f"  {v['name']:<40} mAP@50={m['mAP_50']:.4f}  mAP@50:95={m['mAP_50_95']:.4f}  AP-S={m['mAP_small']:.4f}")

    # Benchmark summary
    bench_file = os.path.join(results_root, gpu_dir, 'benchmark', 'benchmark_results.json')
    if os.path.exists(bench_file):
        with open(bench_file) as f:
            bench = json.load(f)
        print(f"\n\u2501 {gpu_dir} \u2014 Benchmark: {len(bench)} models benchmarked")
        for k, v in bench.items():
            print(f"  {v['name']:<40} {v['mean_latency_ms']:.2f}ms  FPS={v['fps']:.1f}  P95={v['p95_latency_ms']:.2f}ms")

---
## Step 9 — Save Results Back & Download

In [ ]:
#@title 9a. Copy results back to Google Drive (persistent storage)
import shutil

# Save to Drive so results survive Colab session resets
DRIVE_RESULTS_DST = "/content/drive/MyDrive/FYP-Checkpoints/Colab_Results"
src = os.path.join(WORKSPACE, 'eval/results')

if os.path.exists(src):
    if os.path.exists(DRIVE_RESULTS_DST):
        shutil.rmtree(DRIVE_RESULTS_DST)
    shutil.copytree(src, DRIVE_RESULTS_DST)
    print(f"\u2705 Results copied to Google Drive: {DRIVE_RESULTS_DST}")
else:
    print("\u274c No results to copy yet.")

In [ ]:
#@title 9b. Download results as zip to your PC
import shutil
from google.colab import files

zip_path = '/content/eval_results'
results_src = os.path.join(WORKSPACE, 'eval/results')

if os.path.exists(results_src):
    shutil.make_archive(zip_path, 'zip', results_src)
    size_mb = os.path.getsize(zip_path + '.zip') / (1024 * 1024)
    print(f"\u2705 Zipped: {zip_path}.zip ({size_mb:.1f} MB)")
    files.download(f'{zip_path}.zip')
else:
    print("\u274c No results to download yet.")

In [ ]:
#@title 9c. (Optional) Git commit & push new Colab results back to repo
# This pushes the Colab GPU results alongside your existing RTX 4050 results.
# You'll need a GitHub Personal Access Token for private repos.

# os.chdir(WORKSPACE)
# !git config user.email "your-email@example.com"
# !git config user.name "Your Name"
# !git add eval/results/
# !git commit -m "Add Colab T4 GPU evaluation results"
# !git push origin main
# # For private repos, use: git push https://<TOKEN>@github.com/Sasankamadura/FYP-Final-Testing.git main

---
## 💡 Tips & Troubleshooting

### Resume Support
The validation script **automatically skips** models that have already been evaluated.
If your Colab session dies mid-run, just re-run Step 5 — it picks up where it left off.

### Cross-GPU Comparison
Your repo already has results from `GPU_NVIDIA_GeForce_RTX_4050_Laptop_GPU`.
After running on Colab (T4), the report generator will automatically produce a
**cross-GPU FPS comparison** table.

### Performance Tips
| Tip | Details |
|-----|---------|
| **Faster I/O** | Checkpoints are already copied to local `/content/` disk (not read from Drive during eval) |
| **Single model test** | Use `--model baseline_rtdetr_r18` flag to test one model quickly |
| **E2E benchmark** | Add `--e2e` flag to also measure full pipeline (preprocess + inference + postprocess) |
| **Colab Pro** | A100 GPU gives ~3–5x faster benchmarks than T4 |

### Common Issues
| Issue | Fix |
|-------|-----|
| `CUDAExecutionProvider not available` | Runtime → Change runtime type → GPU |
| `FileNotFoundError` on `.onnx` file | Check Step 3b — `DRIVE_CHECKPOINTS` path must match your Drive folder |
| `Session crashed` during validation | Re-run Step 5 (resume support will skip completed models) |
| `git clone` fails (private repo) | Use `!git clone https://<TOKEN>@github.com/Sasankamadura/FYP-Final-Testing.git` |